<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Inference

In [1]:
import os
import sys
import timeit

import numpy as np
import pandas as pd
import tensorflow as tf

from deepcell.applications import NuclearSegmentation
from deepcell_tracking.trk_io import load_trks, save_trk
from deepcell_tracking.tracking import CellTracker as _CellTracker
from deepcell_tracking.tracking import linear_sum_assignment

sys.path.append('/notebooks/benchmarking')
import utils

In [2]:
# Add timers to CellTracker
class CellTracker(_CellTracker):

    def __init__(self, X, y, model, neighborhood_encoder, track_length):
        # Add timers
        self.timer_ne = 0
        self.timer_inf = 0
        self.timer_lap = 0
        
        super().__init__(X, y, model, neighborhood_encoder=neighborhood_encoder, track_length=track_length)

    def _get_neighborhood_embeddings(self, **kwargs):
        tic = timeit.default_timer()
        embeddings = super()._get_neighborhood_embeddings(**kwargs)
        toc = timeit.default_timer()

        self.timer_ne += toc - tic

        return embeddings

    def _track_frame(self, frame):
        tic = timeit.default_timer()
        cost_matrix, predictions = self._get_cost_matrix(frame)
        toc = timeit.default_timer()

        self.timer_inf += toc - tic

        tic = timeit.default_timer()
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        assignments = np.stack([row_ind, col_ind], axis=1)
        self._update_tracks(assignments, frame, predictions)
        toc = timeit.default_timer()

        self.timer_lap += toc - tic


In [3]:
source_data = '/notebooks/data/tracking/test.trks'
        
model_urls = {
    'NuclearSegmentation': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearSegmentation-75.tar.gz',
    'NuclearTrackingNE': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearTrackingNE-75.tar.gz',
    'NuclearTrackingInf': 'https://deepcell-data.s3-us-west-1.amazonaws.com/saved-models/NuclearTrackingInf-75.tar.gz'
}

In [4]:
# Load test data
data = load_trks(source_data)

In [5]:
# Download and load each model
models = {}
for m, url in model_urls.items():
    archive_path = tf.keras.utils.get_file(f'{m}.tgz', url, extract=True, cache_subdir='models')
    model_path = os.path.splitext(archive_path)[0]
    model = tf.keras.models.load_model(model_path)
    models[m] = model

450641920/450633493 [==============================] - 9s 0us/step


2023-06-02 21:27:03.378048: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-02 21:27:03.790551: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46711 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:41:00.0, compute capability: 8.6


548864/538726 [==============================] - 0s 0us/step


In [6]:
# Load segmentation and tracking applications
app_seg = NuclearSegmentation(models['NuclearSegmentation'])

In [7]:
timers = []
for it in range(3):
    for batch_no in range(len(data['lineages'])):
        print(batch_no)
        tm = {'batch': batch_no, 'iteration': it}

        # Pull out relevant data for this batch
        X = data['X'][batch_no]
        y = data['y'][batch_no]
        lineage = data['lineages'][batch_no]

        # Correct discontiguous tracks, which are not allowed by CTC
        y, lineage = utils.convert_to_contiguous(y, lineage)

        # Determine position of zero padding for removal
        slc = utils.find_zero_padding(X)
        X = X[slc]
        y = y[slc]

        # Determine which frames are zero padding
        frames = np.sum(y, axis=(1,2)) # True if image not blank
        good_frames = np.where(frames)[0]
        X = X[:len(good_frames)]
        y = y[:len(good_frames)]

        # Generate tracks on predicted segmentations
        tic = timeit.default_timer()
        y_pred = app_seg.predict(X)
        tm['seg'] = timeit.default_timer() - tic

        tracker = CellTracker(
            X,
            y_pred,
            models['NuclearTrackingInf'],
            neighborhood_encoder=models['NuclearTrackingNE'],
            track_length=8
        )
        tracker.track_cells()
        tm['ne'] = tracker.timer_ne
        tm['inf'] = tracker.timer_inf
        tm['lap'] = tracker.timer_lap

        tm['nframes'] = X.shape[0]
        tm['nobjects'] = sum([len(np.unique(y[t])) - 1 for t in range(y.shape[0])])

        timers.append(tm)

0


2023-06-02 21:27:47.178165: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2023-06-02 21:28:18.571852: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11


In [8]:
df = pd.DataFrame(timers)
df.to_csv('speed.csv')